Distributed Evolutionary Algorithms in Python

https://deap.readthedocs.io/en/master/

In [ ]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 2.7 MB/s eta 0:00:00


In [ ]:
import gdown

# URL do arquivo no Google Drive
url = 'https://drive.google.com/uc?id=1ljr5-z2BlPWJuIsaO_3Mu0n3HYBCiUkM'

# Caminho onde o arquivo será salvo no Colab
output = '/content/knapsack_products.csv'

# Baixando o arquivo
gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1ljr5-z2BlPWJuIsaO_3Mu0n3HYBCiUkM
To: /content/knapsack_products.csv
100%|██████████| 33.7k/33.7k [00:00<00:00, 28.7MB/s]


'/content/knapsack_products.csv'

In [ ]:
import csv, random, argparse, os
from deap import base, creator, tools, algorithms

# Função para carregar itens do CSV
def load_items(csv_path):
    items = []
    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            items.append({
                "id": int(row["id"]),
                "name": row["name"],
                "category": row["category"],
                "weight": int(row.get("weight",0.0)),
                "value": float(row.get("value",0.0)),
                "volume": float(row.get("volume", 0.0)),
            })
    return items

# Função de avaliação (fitness penalizado)
def make_eval(items, capacity, capacity_vol, eps=1e-6):
    n = len(items)
    # cálculo das penalidades λw e λu
    lambda_w = 2.0 * sum(it["value"] / max(it["weight"], eps) for it in items) / n
    lambda_u = 4.0 * sum(it["value"] / max(it["volume"], eps) for it in items) / n

    def eval_knapsack(individual):
        total_w, total_v, total_vol = 0.0, 0.0, 0.0
        for gene, it in zip(individual, items):
            if gene:
                total_w += it["weight"]
                total_v += it["value"]
                total_vol += it["volume"]

        # penalizações caso ultrapasse peso ou volume
        penalty_w = lambda_w * max(0, total_w - capacity)
        penalty_u = lambda_u * max(0, total_vol - capacity_vol)

        if total_w <= capacity and total_vol <= capacity_vol:
           bonus_vol = total_v * (total_vol / capacity_vol)
        else:
             bonus_vol = 0.0

        f = total_v - penalty_w - penalty_u + bonus_vol
        return (max(f, 1e-9),)
    return eval_knapsack

# Função decode para extrair solução

def decode(individual, items):
    chosen = []
    total_w, total_v, total_vol = 0.0, 0.0, 0.0
    for gene, it in zip(individual, items):
        if gene:
            chosen.append(it)
            total_w += it["weight"]
            total_v += it["value"]
            total_vol += it["volume"]
    return chosen, total_w, total_v, total_vol



In [ ]:
import csv, random, os
from deap import base, creator, tools, algorithms

# Hiperparâmetros
CAPACITY     = 500
CAPACITY_VOL = 200
POP          = 300
GEN          = 200
CXPB         = 0.7
MUTPB        = 0.08
TOURNSIZE    = 3
CSV_PATH     = "/content/knapsack_products.csv"
OUT_DIR      = "/content/resultados_por_semente"
os.makedirs(OUT_DIR, exist_ok=True)

# Carregar itens
items = load_items(CSV_PATH)

# Criar classes DEAP
try:
    del creator.FitnessMax
    del creator.Individual
except Exception:
    pass
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.choices, [0,1], weights=[0.95, 0.05], k=1)
toolbox.register("individual", tools.initRepeat, creator.Individual, lambda: toolbox.attr_bool()[0], n=len(items))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=MUTPB)
toolbox.register("select", tools.selTournament, tournsize=TOURNSIZE)

eval_func = make_eval(items, CAPACITY, CAPACITY_VOL)
toolbox.register("evaluate", eval_func)

# Lista de sementes
seeds = [1,5,42,999,13]
results = []

# Loop das sementes
for SEED in seeds:
    random.seed(SEED)
    pop = toolbox.population(n=POP)
    hof = tools.HallOfFame(1)

    stats = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats.register("avg", lambda fits: sum(fits)/len(fits))
    stats.register("max", max)

    pop, log = algorithms.eaSimple(
        pop, toolbox, cxpb=CXPB, mutpb=MUTPB,
        ngen=GEN, stats=stats, halloffame=hof, verbose=False
    )

    best = hof[0]
    chosen, total_w, total_v, total_vol = decode(best, items)



    # Salvar CSV da semente atual
    seed_csv = f"{OUT_DIR}/itens_seed_{SEED}.csv"
    with open(seed_csv, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["id", "name", "category", "weight", "value", "volume"])
        for it in chosen:
            writer.writerow([it["id"], it["name"], it["category"], it["weight"], it["value"], it["volume"]])
        writer.writerow([])
        writer.writerow(["Total Weight", total_w])
        writer.writerow(["Total Volume", total_vol])
        writer.writerow(["Total Value", total_v])

    results.append({
        "seed": SEED,
        "best_value": total_v,
        "total_weight": total_w,
        "total_volume": total_vol,
        "selected_items": len(chosen)
    })

# Salvar CSV com o resumo final de todas as sementes
OUT_CSV = f"{OUT_DIR}/resumo_sementes.csv"
with open(OUT_CSV, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["seed","best_value","total_weight","total_volume","selected_items"])
    writer.writeheader()
    for r in results:
        writer.writerow(r)

print(f"✅ Resultados individuais salvos em: {OUT_DIR}")
print(f"✅ Resumo geral: {OUT_CSV}")





✅ Resultados individuais salvos em: /content/resultados_por_semente
✅ Resumo geral: /content/resultados_por_semente/resumo_sementes.csv


In [ ]:
import pandas as pd
df = pd.read_csv(OUT_CSV)

In [ ]:
df

,seed,best_value,total_weight,total_volume,selected_items
0,1,8498.42,499.0,183.13,69
1,5,8236.87,497.0,198.77,78
2,42,8144.78,500.0,197.44,75
3,999,8520.30,500.0,199.27,74
4,13,8557.56,500.0,197.39,79
